GCN + PDG

In [6]:
import os
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import json
import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def assign_node_type(node):
    node = node.strip()
    if 'if' in node:
        return 'condition'
    elif 'for' in node or 'while' in node:
        return 'loop'
    elif 'System.out.println' in node:
        return 'output'
    elif node.startswith('//') or node == '' or 'LogFrame' in node:
        return 'meaningless'
    return 'statement'

def assign_edge_type(edge):
    return 'control' if any(keyword in edge[0] for keyword in ['if', 'for', 'while']) else 'data'

def generate_pdg(code):
    G = nx.DiGraph()
    lines = code.splitlines()
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        node_type = assign_node_type(line)
        if node_type != 'meaningless':
            G.add_node(i, label=line, type=node_type)
            if i > 0:
                prev_line = lines[i-1].strip()
                prev_node_type = assign_node_type(prev_line)
                if prev_node_type != 'meaningless':
                    G.add_edge(i-1, i, type=assign_edge_type((prev_line, line)))

    isolated_nodes = [node for node, degree in G.degree if degree == 0]
    G.remove_nodes_from(isolated_nodes)
    mapping = {node: idx for idx, node in enumerate(G.nodes())}
    return nx.relabel_nodes(G, mapping) if G.number_of_nodes() > 0 else None

def convert_nx_to_torch_data(G):
    node_features = torch.tensor(
        [[int(node_data['type'] == t) for t in ['condition', 'loop', 'output', 'statement']] for _, node_data in G.nodes(data=True)],
        dtype=torch.float
    )
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    return Data(x=node_features, edge_index=edge_index)

def create_pdg_pairs(data_file, jsonl_file):
    with open(jsonl_file, 'r') as f:
        url_to_code = {entry['idx']: entry['func'] for entry in (json.loads(line.strip()) for line in f)}

    pdg_pairs, true_labels = [], []
    with open(data_file, 'r') as f:
        for line in tqdm(f, desc="Creating PDG pairs"):
            url1, url2, label = line.strip().split('\t')
            if url1 not in url_to_code or url2 not in url_to_code:
                continue

            pdg1, pdg2 = generate_pdg(url_to_code[url1]), generate_pdg(url_to_code[url2])
            if pdg1 is None or pdg2 is None:
                continue

            pdg_pairs.append((convert_nx_to_torch_data(pdg1), convert_nx_to_torch_data(pdg2)))
            true_labels.append(int(label))  # 0, 1, 2, 3 for Type-1, Type-2, Type-3, Type-4 respectively

    return pdg_pairs, true_labels

# Save checkpoint function
def save_checkpoint(model, optimizer, epoch, f1_score, accuracy, precision, recall, metrics, output_dir):
    checkpoint_path = os.path.join(output_dir, f"model_epoch_{epoch}_f1_{f1_score:.4f}.pth")
    
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': {
            'best_f1': f1_score,
            'best_accuracy': accuracy,
            'best_precision': precision,
            'best_recall': recall,
            'per_type_metrics': metrics  
        }
    }, checkpoint_path)
    
    print(f"Checkpoint saved to {checkpoint_path}")

class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.3):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

class CloneDetector(nn.Module):
    def __init__(self, gnn):
        super(CloneDetector, self).__init__()
        self.gnn = gnn
        self.fc = nn.Linear(256, 4)  # 4 classes for Type-1, Type-2, Type-3, Type-4

    def forward(self, data):
        x1, edge_index1, batch1 = data[0].x, data[0].edge_index, data[0].batch
        x2, edge_index2, batch2 = data[1].x, data[1].edge_index, data[1].batch
        
        emb1 = self.gnn(x1, edge_index1, batch1)
        emb2 = self.gnn(x2, edge_index2, batch2)
        
        combined = torch.cat([emb1, emb2], dim=1)
        output = self.fc(combined)
        return output

def train(model, train_loader, valid_loader, epochs=20, output_dir="checkpoints"):
    optimizer = Adam(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=10, verbose=True)
    criterion = nn.CrossEntropyLoss()
    best_f1 = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            graph_pair, labels = batch
            graph_pair = [g.to(device) for g in graph_pair]
            labels = labels.to(device)
            
            optimizer.zero_grad()
            logits = model(graph_pair)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        val_loss, val_metrics = evaluate(model, valid_loader)
        val_f1 = val_metrics['overall']['f1']
        val_accuracy = val_metrics['overall']['accuracy']
        val_precision = val_metrics['overall']['precision']
        val_recall = val_metrics['overall']['recall']
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}')
        
        scheduler.step(val_f1)
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            save_checkpoint(model, optimizer, epoch, best_f1, val_accuracy, 
                val_precision, val_recall, val_metrics, output_dir)

def evaluate(model, data_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            graph_pair, labels = batch
            graph_pair = [g.to(device) for g in graph_pair]
            labels = labels.to(device)
            
            logits = model(graph_pair)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds, all_labels = np.array(all_preds), np.array(all_labels)

    metrics = {'overall': {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='weighted', zero_division=0),
        'f1': f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    }}

    for clone_type in range(4):
        type_preds, type_labels = (all_preds == clone_type), (all_labels == clone_type)
        metrics[f'Type-{clone_type+1}'] = {
            'accuracy': accuracy_score(type_labels, type_preds),
            'precision': precision_score(type_labels, type_preds, zero_division=0),
            'recall': recall_score(type_labels, type_preds, zero_division=0),
            'f1': f1_score(type_labels, type_preds, zero_division=0)
        }

    return total_loss / len(data_loader), metrics

# Main execution logic
input_dim = 4
hidden_dim = 64
output_dim = 128
dropout_rate = 0.3
gnn = GNN(input_dim, hidden_dim, output_dim, dropout_rate)
model = CloneDetector(gnn).to(device)

train_file, valid_file = 'test_set/train_5000.txt', 'test_set/train_5000.txt'
jsonl_file = 'test_set/data.jsonl'

train_pdg_pairs, train_labels = create_pdg_pairs(train_file, jsonl_file)
valid_pdg_pairs, valid_labels = create_pdg_pairs(valid_file, jsonl_file)

def collate_fn(batch):
    graphs, labels = zip(*batch)
    batched_graphs = [Batch.from_data_list([g[0] for g in graphs]),
                      Batch.from_data_list([g[1] for g in graphs])]
    return batched_graphs, torch.tensor(labels)

train_loader = DataLoader(list(zip(train_pdg_pairs, train_labels)), batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(list(zip(valid_pdg_pairs, valid_labels)), batch_size=64, collate_fn=collate_fn)

output_dir = "checkpoints_GCN_cosine_similarity"
os.makedirs(output_dir, exist_ok=True)

train(model, train_loader, valid_loader, epochs=100, output_dir=output_dir)

Creating PDG pairs: 3000it [00:02, 1415.54it/s]
Creating PDG pairs: 3000it [00:02, 1350.51it/s]
c:\Users\PC\Desktop\pythonProject1\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Evaluating: 100%|██████████| 23/23 [00:00<00:00, 47.66it/s]


Epoch 1/100, Train Loss: 1.0272, Val Loss: 0.7715, Val F1: 0.6155
Checkpoint saved to checkpoints_GCN_cosine_similarity\model_epoch_0_f1_0.6155.pth


Evaluating: 100%|██████████| 23/23 [00:00<00:00, 60.52it/s]


Epoch 2/100, Train Loss: 0.7748, Val Loss: 0.7202, Val F1: 0.7506
Checkpoint saved to checkpoints_GCN_cosine_similarity\model_epoch_1_f1_0.7506.pth


Evaluating: 100%|██████████| 23/23 [00:00<00:00, 57.68it/s]


...

Epoch 98/100, Train Loss: 0.6155, Val Loss: 0.6008, Val F1: 0.8198


Evaluating: 100%|██████████| 23/23 [00:00<00:00, 57.70it/s]


Epoch 99/100, Train Loss: 0.6161, Val Loss: 0.6008, Val F1: 0.8198


Evaluating: 100%|██████████| 23/23 [00:00<00:00, 68.09it/s]

Epoch 100/100, Train Loss: 0.6135, Val Loss: 0.6008, Val F1: 0.8198


In [7]:
checkpoint = torch.load('checkpoints_GCN_cosine_similarity/model_epoch_17_f1_0.8482.pth')
per_type_metrics = checkpoint['metrics']['per_type_metrics']

# Print overall metrics first
overall_metrics = per_type_metrics['overall']
print("Overall metrics:")
print(f"Accuracy-score: {overall_metrics['accuracy']:.4f}")
print(f"Precision-score: {overall_metrics['precision']:.4f}")
print(f"Recall-score: {overall_metrics['recall']:.4f}")
print(f"F1-score: {overall_metrics['f1']:.4f}")

# Print per-type metrics
for clone_type, metrics in per_type_metrics.items():
    if clone_type != 'overall':  # Skip overall metrics since already printed
        print(f"\n{clone_type}:")
        print(f"Accuracy-score: {metrics['accuracy']:.4f}")
        print(f"Precision-score: {metrics['precision']:.4f}")
        print(f"Recall-score: {metrics['recall']:.4f}")
        print(f"F1-score: {metrics['f1']:.4f}")



Overall metrics:
Accuracy-score: 0.8519
Precision-score: 0.8536
Recall-score: 0.8519
F1-score: 0.8482

Type-1:
Accuracy-score: 0.9939
Precision-score: 0.9000
Recall-score: 0.5294
F1-score: 0.6667

Type-2:
Accuracy-score: 0.9256
Precision-score: 0.8895
Recall-score: 0.6576
F1-score: 0.7562

Type-3:
Accuracy-score: 0.9024
Precision-score: 0.8589
Recall-score: 0.9439
F1-score: 0.8994

Type-4:
Accuracy-score: 0.8819
Precision-score: 0.8273
Recall-score: 0.8385
F1-score: 0.8329
